# Rapprochement DPE juillet 2025

In [11]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
usecols = ['numero_dpe', 'identifiant_ban', 'code_postal_ban', 'code_postal_brut', 'id_rnb', 'provenance_id_rnb']
dtype={'numero_dpe': 'string', 'identifiant_ban': 'string', 'code_postal_ban': 'string', 'code_postal_brut': 'string', 'id_rnb': 'string', 'provenance_id_rnb': 'string'}

# usecols = ['numero_dpe', 'id_rnb', 'provenance_id_rnb']
# dtype={'numero_dpe': 'string', 'id_rnb': 'string', 'provenance_id_rnb': 'string'}

In [ ]:
df_tertiaire = pd.read_csv('notebooks/rapprochements/DPE/2025/data/dpe01tertiaire.csv', sep=',', usecols=usecols, dtype=dtype)
df_tertiaire['file'] = 'tertiaire'

In [5]:
df_existant = pd.read_csv('notebooks/rapprochements/DPE/2025/data/dpe03existant.csv', sep=',', usecols=usecols, dtype=dtype)
df_existant['file'] = 'existant'

In [4]:
df_neuf = pd.read_csv('notebooks/rapprochements/DPE/2025/data/dpe02neuf.csv', sep=',', usecols=usecols, dtype=dtype)
df_neuf['file'] = 'neuf'

In [5]:
df = pd.concat([df_tertiaire, df_neuf, df_existant])

In [6]:
df.shape

(13742029, 7)

In [7]:
df.head()

,numero_dpe,id_rnb,provenance_id_rnb,code_postal_ban,identifiant_ban,code_postal_brut,file
0,2269T2615953W,<NA>,<NA>,69440,69228_0086_00707,69440,tertiaire
1,2313T1646022S,<NA>,<NA>,13360,13086_0035_00008,13360,tertiaire
2,2404T3934296Y,<NA>,<NA>,04220,04197_0231,04220,tertiaire
3,2394T2686905U,<NA>,<NA>,94420,94059_0530_00165,94420,tertiaire
4,2360T0127880L,<NA>,<NA>,60650,60073_0075_00007,60650,tertiaire


In [ ]:
# df_tertiaire_sample = df_tertiaire[df_tertiaire['id_rnb'].notnull()].sample(20)
# df_tertiaire_sample

# df_tertiaire_rnb = df_tertiaire.groupby('provenance_id_rnb').count()
# df_tertiaire_rnb

# df_existant_rnb = df_existant.groupby('provenance_id_rnb', dropna=False).count()
# df_existant_rnb

# df_tertiaire_rnb_logiciel = df_tertiaire[df_tertiaire['provenance_id_rnb'] == 'Logiciel']
# df_tertiaire_rnb_logiciel.shape

# df_existant_rnb_logiciel = df_existant[df_existant['provenance_id_rnb'] == 'Logiciel']
# df_existant_rnb_logiciel.shape

(398, 7)

In [ ]:
# create 100 subfiles in the dpe_existant folder, using the number of rows in the dataframe
for i in range(0, 100):
    df.iloc[i*df.shape[0]//100:(i+1)*df.shape[0]//100].to_csv(f'notebooks/rapprochements/DPE/2025/sub_files/dpe-{i}.csv', index=False)

: 

In [ ]:
# le rapprochement

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from concurrent.futures import ThreadPoolExecutor
import numpy as np

def get_rnb_id(row):
    cursor = connection.cursor()

    if row['code_postal_ban'] != row['code_postal_brut']:
        return []
    
    ban_id = row['identifiant_ban']

    sql = f"""
            with rnb_ids as (
            select
                rnb_id
            from
                batid_buildingaddressesreadonly bb
            left join batid_building bb2 on
                bb2.id = bb.building_id
            where
                address_id = '{ban_id}'
                and ST_AREA(shape::geography) > 25)
            select
                array_agg(rnb_id)
            from
                rnb_ids;
    """
    cursor.execute(sql)
    result = cursor.fetchone()
    return result[0] if result[0] is not None else []

def execute(df):
    df_copy = df.copy()
    df_copy['rnb_id_rappro'] = df_copy.apply(get_rnb_id, axis=1)
    return df_copy


def process_sub_file(i):
    print(f"processing file {i}")
    if not os.path.exists(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv'):
        df_sub_file = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files/dpe-{i}.csv', sep=',')
        max_workers = 50
        dfs = np.array_split(df_sub_file, max_workers)

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results_sub_file = executor.map(execute, dfs)
            df_result = pd.concat(results_sub_file)
            df_result.to_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', index=False)

for i in range(100):
    process_sub_file(i)

processing file 0
processing file 1
processing file 2
processing file 3
processing file 4
processing file 5
processing file 6
processing file 7
processing file 8
processing file 9
processing file 10
processing file 11
processing file 12
processing file 13
processing file 14
processing file 15
processing file 16
processing file 17
processing file 18
processing file 19
processing file 20
processing file 21
processing file 22
processing file 23
processing file 24
processing file 25
processing file 26
processing file 27
processing file 28
processing file 29
processing file 30
processing file 31
processing file 32
processing file 33
processing file 34
processing file 35
processing file 36
processing file 37
processing file 38
processing file 39
processing file 40
processing file 41
processing file 42
processing file 43
processing file 44
processing file 45
processing file 46
processing file 47
processing file 48
processing file 49
processing file 50
processing file 51
processing file 52
pro

In [ ]:
# aggregation des sous fichiers de résultats en un seul
dfs_result = []

for i in range(100):
    df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
    dfs_result.append(df_result)

df_final = pd.concat(dfs_result)
df_final['rnb_id_rappro'] = df_final['rnb_id_rappro'].apply(eval)


/tmp/ipykernel_2074/3264209841.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
/tmp/ipykernel_2074/3264209841.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
/tmp/ipykernel_2074/3264209841.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
/tmp/ipykernel_2074/3264209841.py:4: Dty

In [ ]:
# sauvegarde des résultats
df_final.to_csv('notebooks/rapprochements/DPE/2025/results_DPE_RNB.csv', index=False)

In [39]:
df_final = pd.read_csv('notebooks/rapprochements/DPE/2025/results_DPE_RNB.csv')

/tmp/ipykernel_2430/4251158513.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('notebooks/rapprochements/DPE/2025/results_DPE_RNB.csv')


In [40]:
df_final.shape

(13742029, 5)

In [4]:
df_final.head()

,numero_dpe,id_rnb,provenance_id_rnb,file,rnb_id_rappro
0,2269T2615953W,NaN,NaN,tertiaire,"['4YHBR2QC67QF', 'ESW1SCP3YRKF', 'QZW45SDXBJDF..."
1,2313T1646022S,NaN,NaN,tertiaire,"['9FFMHKTEQM2A', '73WPY3ZD4A3Y', 'KNT9AXYTFQVV']"
2,2404T3934296Y,NaN,NaN,tertiaire,[]
3,2394T2686905U,NaN,NaN,tertiaire,"['FQMXR865P7AX', 'AS59FP2BQ16B']"
4,2360T0127880L,NaN,NaN,tertiaire,"['SV4KCWDCK333', 'K37C6J5ZA9FT', '386R58WKYE81']"


In [ ]:
# stats sur la provenance des ID-RNB dans le fichier
df_gb = df_final.groupby(['file', 'provenance_id_rnb'], dropna=False)['numero_dpe'].count()
df_gb

file       provenance_id_rnb
existant   Logiciel               21363
           Reprise RNB          5536208
           NaN                  6637391
neuf       Logiciel                 297
           Reprise RNB           221807
           NaN                   908056
tertiaire  Logiciel                 398
           Reprise RNB           157816
           NaN                   258693
Name: numero_dpe, dtype: int64

In [41]:
df_logiciel = df_final[df_final['provenance_id_rnb'] == 'Logiciel']
df_logiciel.shape

(22058, 5)

In [ ]:
# ajout de colonnes pour différencier les différents cas possibles

# diagnostiqueur et rappro RNB sont d'accord
df_logiciel['match'] = df_logiciel.apply(
    lambda row: len(eval(row['rnb_id_rappro'])) == 1 and eval(row['rnb_id_rappro'])[0] == row['id_rnb'],
    axis=1
)

# diagnostiqueur communique un ID-RNB, nous en sommes incapables
df_logiciel['diag_more_precise'] = df_logiciel.apply(
    lambda row: len(eval(row['rnb_id_rappro'])) != 1,
    axis=1
)

# diagnostiqueur et rappro RNB trouvent un ID-RNB unique mais nous ne sommes pas d'accord
df_logiciel['error_rappro'] = df_logiciel.apply(
    lambda row: len(eval(row['rnb_id_rappro'])) == 1 and eval(row['rnb_id_rappro'])[0] != row['id_rnb'],
    axis=1
)

df_logiciel.head(20)

/tmp/ipykernel_2430/1657396994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_logiciel['match'] = df_logiciel.apply(
/tmp/ipykernel_2430/1657396994.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_logiciel['diag_more_precise'] = df_logiciel.apply(
/tmp/ipykernel_2430/1657396994.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,numero_dpe,id_rnb,provenance_id_rnb,file,rnb_id_rappro,match,diag_more_precise,error_rappro
158109,2502T1068524X,11WWDS93XWSQ,Logiciel,tertiaire,"['5R7G3G7V51V1', '11WWDS93XWSQ']",False,True,False
158208,2594T1020535S,GCAEFW4KXAZ7,Logiciel,tertiaire,['GCAEFW4KXAZ7'],True,False,False
158291,2584T1189899I,RCAJ157D6ZDT,Logiciel,tertiaire,[],False,True,False
158349,2594T1168780L,GCAEFW4KXAZ7,Logiciel,tertiaire,['GCAEFW4KXAZ7'],True,False,False
162850,2520T0590415U,T76RJ86W2WFG,Logiciel,tertiaire,"['751SB65D2669', '8B58SJVA7VCP', '8HW3M67XC263...",False,True,False
165033,2484T3785846K,K9V8XF2WGTBR,Logiciel,tertiaire,"['5XXGCHNRSR6J', 'GBFSPPV13RVS', '11NE32VJK2CS...",False,True,False
166687,2584T0853814Z,1D1CPKXM4K1R,Logiciel,tertiaire,[],False,True,False
168453,2533T0613268G,5DZMV3PRZ4M9,Logiciel,tertiaire,['5DZMV3PRZ4M9'],True,False,False
173020,2584T0467096F,YBWQ1M2ZH2HN,Logiciel,tertiaire,[],False,True,False
180671,2584T0444616P,GBDBVBMYNXVZ,Logiciel,tertiaire,['GBDBVBMYNXVZ'],True,False,False


In [ ]:
# stats sur les différents cas
df_logiciel.groupby(['file', 'match', 'diag_more_precise', 'error_rappro'])['match'].count()

file       match  diag_more_precise  error_rappro
existant   False  False              True             2156
                  True               False           10052
           True   False              False            9155
neuf       False  False              True               71
                  True               False             188
           True   False              False              38
tertiaire  False  False              True               35
                  True               False             213
           True   False              False             150
Name: match, dtype: int64

### Etude pourquoi des erreurs sur certains cas (erreur_rappro)

In [ ]:
# on sélectionne toutes les lignes de désaccord
df_error_rappro = df_logiciel[df_logiciel['error_rappro']]
df_error_rappro.shape


(2262, 8)

In [ ]:
# on tire un échantillon de 100 lignes
df_error_rappro_sample = df_error_rappro.sample(100)
df_error_rappro_sample.head(5)

,numero_dpe,id_rnb,provenance_id_rnb,file,rnb_id_rappro,match,diag_more_precise,error_rappro
13510645,2577E1857492D,FTK7WTFD2YQJ,Logiciel,existant,['RA2KSWKJCCW2'],False,False,True
12258927,2556E1206580G,54PGG4Q7WHB2,Logiciel,existant,['HWXE6EBMQGW1'],False,False,True
13255164,2538E1468775Z,1HE6PDVV1ZM7,Logiciel,existant,['RSYZ279TMHE2'],False,False,True
13459697,2534E1768005R,FMS4KDMXXG1W,Logiciel,existant,['BECZA8A7JC9M'],False,False,True
10634111,2469E3298295H,GTHSH6V7Z9AV,Logiciel,existant,['117HGJQD4H7H'],False,False,True


In [45]:
df_error_rappro_sample.to_csv('/app/notebooks/rapprochements/DPE/2025/error_rappro_sample_2.csv', index=False)

In [46]:
df_error_rappro_sample

,numero_dpe,id_rnb,provenance_id_rnb,file,rnb_id_rappro,match,diag_more_precise,error_rappro
13510645,2577E1857492D,FTK7WTFD2YQJ,Logiciel,existant,['RA2KSWKJCCW2'],False,False,True
12258927,2556E1206580G,54PGG4Q7WHB2,Logiciel,existant,['HWXE6EBMQGW1'],False,False,True
13255164,2538E1468775Z,1HE6PDVV1ZM7,Logiciel,existant,['RSYZ279TMHE2'],False,False,True
13459697,2534E1768005R,FMS4KDMXXG1W,Logiciel,existant,['BECZA8A7JC9M'],False,False,True
10634111,2469E3298295H,GTHSH6V7Z9AV,Logiciel,existant,['117HGJQD4H7H'],False,False,True
12826410,2526E0815757L,11M84G3FFT7W,Logiciel,existant,['3ARPPZ96BHH7'],False,False,True
12617723,2534E0166538U,XVW5QFV917YF,Logiciel,existant,['5TPQ4KJA5ANF'],False,False,True
13625291,2524E1988302G,F9WJQGB25XJ6,Logiciel,existant,['5N7TCPH1J895'],False,False,True
10130322,2474E3977272Z,X6C17K3R5RPK,Logiciel,existant,['52YRFAHDAPNS'],False,False,True
12756300,2459E3542456S,AB286MP6QTHX,Logiciel,existant,['7DEF6B6EY5VE'],False,False,True


In [47]:
import requests

def get_infos_from_dpe_api(row):
    # print(row)
    id_dpe = row['numero_dpe']
    res = requests.get(f"https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines?q={id_dpe}")
    data = res.json()['results']
    if len(data) > 0:
        data = data[0]
        infos = pd.Series({'adresse_ban': data['adresse_ban'], 'date_etablissement_dpe': data['date_etablissement_dpe'], 'nom_commune_brut': data['nom_commune_brut'], 'adresse_brut': data['adresse_brut']})
        return infos




In [ ]:
# ajout de quelques infos pour faciliter la vérification humaine
df_error_rappro_sample = pd.read_csv('/app/notebooks/rapprochements/DPE/2025/error_rappro_sample_2.csv')
df_error_rappro_sample['batiment_diagnostiqueur'] = df_error_rappro_sample.apply(lambda row: f'https://rnb.beta.gouv.fr/carte?q={row['id_rnb']}', axis=1)
df_error_rappro_sample['batiment_rapprochement'] = df_error_rappro_sample.apply(lambda row: f'https://rnb.beta.gouv.fr/carte?q={eval(row['rnb_id_rappro'])[0]}', axis=1)
df_error_rappro_sample[["adresse_ban","date_etablissement_dpe","nom_commune_brut","adresse_brut"]] = df_error_rappro_sample.apply(get_infos_from_dpe_api, axis=1)
df_error_rappro_sample


,numero_dpe,id_rnb,provenance_id_rnb,file,rnb_id_rappro,match,diag_more_precise,error_rappro,batiment_diagnostiqueur,batiment_rapprochement,adresse_ban,date_etablissement_dpe,nom_commune_brut,adresse_brut
0,2577E1857492D,FTK7WTFD2YQJ,Logiciel,existant,['RA2KSWKJCCW2'],False,False,True,https://rnb.beta.gouv.fr/carte?q=FTK7WTFD2YQJ,https://rnb.beta.gouv.fr/carte?q=RA2KSWKJCCW2,10 Rue Roland Garros 77178 Saint-Pathus,2025-06-04,SAINT-PATHUS,10 rue Roland Garros
1,2556E1206580G,54PGG4Q7WHB2,Logiciel,existant,['HWXE6EBMQGW1'],False,False,True,https://rnb.beta.gouv.fr/carte?q=54PGG4Q7WHB2,https://rnb.beta.gouv.fr/carte?q=HWXE6EBMQGW1,60 Rue Madame Molé 56000 Vannes,2025-04-07,Vannes,56-58-60 Rue Madame Molé
2,2538E1468775Z,1HE6PDVV1ZM7,Logiciel,existant,['RSYZ279TMHE2'],False,False,True,https://rnb.beta.gouv.fr/carte?q=1HE6PDVV1ZM7,https://rnb.beta.gouv.fr/carte?q=RSYZ279TMHE2,12 Avenue de Grenoble 38170 Seyssinet-Pariset,2025-04-29,Seyssinet-Pariset,12 Avenue de Grenoble
3,2534E1768005R,FMS4KDMXXG1W,Logiciel,existant,['BECZA8A7JC9M'],False,False,True,https://rnb.beta.gouv.fr/carte?q=FMS4KDMXXG1W,https://rnb.beta.gouv.fr/carte?q=BECZA8A7JC9M,75 Rue Jacques-Louis David 34070 Montpellier,2025-05-27,Montpellier,75 Rue Jacques-Louis David
4,2469E3298295H,GTHSH6V7Z9AV,Logiciel,existant,['117HGJQD4H7H'],False,False,True,https://rnb.beta.gouv.fr/carte?q=GTHSH6V7Z9AV,https://rnb.beta.gouv.fr/carte?q=117HGJQD4H7H,8 Rue Alexandre Berthier 69110 Sainte-Foy-lès-...,2024-09-10,Sainte-Foy-lès-Lyon,8 Rue Alexandre Berthier
5,2526E0815757L,11M84G3FFT7W,Logiciel,existant,['3ARPPZ96BHH7'],False,False,True,https://rnb.beta.gouv.fr/carte?q=11M84G3FFT7W,https://rnb.beta.gouv.fr/carte?q=3ARPPZ96BHH7,56 Allee du Chateau 26560 Montfroc,2025-03-10,MONTFROC,56 allée du Chateau
6,2534E0166538U,XVW5QFV917YF,Logiciel,existant,['5TPQ4KJA5ANF'],False,False,True,https://rnb.beta.gouv.fr/carte?q=XVW5QFV917YF,https://rnb.beta.gouv.fr/carte?q=5TPQ4KJA5ANF,11 Rue Jean Vachet Bâtiment E - 34070 Montpel...,2025-01-13,Montpellier,11 Rue Jean Vachet
7,2524E1988302G,F9WJQGB25XJ6,Logiciel,existant,['5N7TCPH1J895'],False,False,True,https://rnb.beta.gouv.fr/carte?q=F9WJQGB25XJ6,https://rnb.beta.gouv.fr/carte?q=5N7TCPH1J895,9 Rue des Farges 24160 Saint-Martial-d'Albarède,2025-06-16,SAINT-MARTIAL-D'ALBARÈDE,9 rue des Farges
8,2474E3977272Z,X6C17K3R5RPK,Logiciel,existant,['52YRFAHDAPNS'],False,False,True,https://rnb.beta.gouv.fr/carte?q=X6C17K3R5RPK,https://rnb.beta.gouv.fr/carte?q=52YRFAHDAPNS,4 Rue Saint-Etienne 74960 Annecy,2024-11-11,CRAN-GEVRIER,4 rue Saint-Etienne
9,2459E3542456S,AB286MP6QTHX,Logiciel,existant,['7DEF6B6EY5VE'],False,False,True,https://rnb.beta.gouv.fr/carte?q=AB286MP6QTHX,https://rnb.beta.gouv.fr/carte?q=7DEF6B6EY5VE,11 Place d'Artois 59150 Wattrelos,2024-10-08,Wattrelos,11 Place d'Artois


In [ ]:
# sauvegarde dans un .csv pour étude
df_error_rappro_sample.to_csv('/app/notebooks/rapprochements/DPE/2025/error_rappro_sample_rich_2.csv', index=False)